# Photosensitization of CP1 by Xan, Delayed fluorescence


Beer–Lambert law

$$A(\lambda) = \varepsilon(\lambda)cl = -\log T(\lambda) =  -\log{\frac{I(\lambda)}{I_0(\lambda)}}$$

where $A(\lambda)$ is absorbance at wavelength $\lambda$, $\varepsilon(\lambda)$ is molar absorption coefficient of compound at wavelenth $\lambda$, $c$ is the concentration of that compound, $l$ is the length of cuvette, $T$ is a transmitance (the part of transmitted light), $I$ is intensity of transmitted light and $I_0$ is intensity of incident light. Beer–Lambert law is additive, so for $n$ absorbing species in the same measured solution, resulting absorbance will be

$$A(\lambda) = l\sum_{i=1}^n c_i\varepsilon_i(\lambda)$$

For time dependent measurement, where concentrations of species change in time, absorbance becomes dependent on time

$$A(\lambda, t) = l\sum_{i=1}^n c_i(t)\varepsilon_i(\lambda)$$

From transient measurement (or any spectroscopic kinetic measurement), we usually obtain a matrix $\overset{t\ \times\ w}{\mathbf{A}}$, where $t$ is number of measured times and $w$ is number of measured wavelengths. We can look at $\mathbf A$ as a column vector of dimension $t$. Then, it represents measured spectra for different times, thus, time evolution.


In [56]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from scipy.integrate import odeint

%matplotlib inline

In [80]:
times = np.linspace(0,2,500)

@widgets.interact(k0=(0, 2, 0.2), k1=(0.5, 30, 2), kd=(0.5, 10, 1),
                  kTT_DA=(0, 1, 0.1), kTT_AA=(0, 1, 0.1), k_delay=(1, 100, 2), No_Delay=True)
def plot_func(k0, k1, kd, kTT_DA, kTT_AA, k_delay, No_Delay):
  
    def solve(conc, t):
        cA, cB, cC = conc

        dcA_dt = -(k0 + k1) * cA
        dcB_dt = + k1 * cA - kd * cB
        dcC_dt = + kTT_DA * cA * cB + kTT_AA * cB ** 2 - k_delay * cC

        return [dcA_dt, dcB_dt, dcC_dt]
    
    result = odeint(solve, [1, 0, 0], times)
    
    A = result[:, 0]
    B = result[:, 1]
    C = result[:, 2] if not No_Delay else kTT_DA * A * B + kTT_AA * B ** 2
    C = C / np.max(C) * 0.3  # normalize delay fluorescence curve to value 0.3
    
    fig, ax = plt.subplots()
    ax.set_xlim(-0.02, 2)
    ax.set_ylim(-0.02, 1.02)
    ax.set_ylabel('$c/c_0$')
    ax.set_xlabel('Time')
    
    plt.rcParams['figure.figsize'] = [15, 8]
    
    ax.plot(times, A , color='r', label='$^3Xan$')
    ax.plot(times, B , color='b', label='$^3DR1$')
    ax.plot(times, C , color='black', ls=':', lw=3, label='Delayed fluorescence-norm.')
    
    ax.legend()

interactive(children=(FloatSlider(value=1.0, description='k0', max=2.0, step=0.2), FloatSlider(value=14.5, des…